<a href="https://colab.research.google.com/github/41371125h-chinrouzhen/114-1-PL/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A%EF%BC%88AI%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **成績一本通**
目標：從 Sheet 讀各次作業/測驗 → 計平時/期末/總成績 → 回寫成績總表。

AI 點子：針對「開放式作答」欄位，用模型產出整體摘要與常見迷思整理（不評分，只做總結）。

Google sheet鏈接:https://docs.google.com/spreadsheets/d/1HA1cvniLmO1O362GtHlrLEK3TQ8_XQJBULMpbaU-29s/edit?usp=sharing

注記：API key不可用，無法生成相關總結


In [3]:
# 安裝 匯入套件與認證
!pip install -q gradio gspread pandas google-generativeai

import pandas as pd
import gspread
from google.colab import auth, userdata
from google.auth import default
import google.generativeai as genai
import gradio as gr

In [4]:
# Google Sheets 認證
gc = None
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ Google Sheets 認證成功。")
except Exception as e:
    print(f"❌ Google Sheets 認證失敗: {e}")

✅ Google Sheets 認證成功。


In [12]:
# Gemini API 認證 (使用 Colab Secrets)
model = None
try:
    api_key = userdata.get('GEMINI_API_KEY')
    if not api_key:
        raise ValueError("在 Colab Secrets 中找不到 'GEMINI_API_KEY'。")
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.5-flash')
    print("✅ Gemini API 已成功設定。")

    # 執行一個簡單的測試來確認 API 是否正常運作
    response = model.generate_content('Hi')
    if response.text:
      print("👍 Gemini API 測試連線成功。")

except Exception as e:
    print(f"❌ Gemini API 設定或測試失敗: {e}")

✅ Gemini API 已成功設定。
👍 Gemini API 測試連線成功。


In [13]:
# AI
def get_student_analysis(sheet_url, student_id):
    """
    主要處理函式：根據 Google Sheet URL 和學號，取得成績並產生 AI 分析。
    """
    if not gc or not model:
        return None, "錯誤：Google Sheets 或 Gemini API 未成功認證，請返回執行第二部分。"

    if not sheet_url or not student_id:
        return None, "錯誤：請務必輸入 Google Sheet 網址和學號。"

    try:
        print(f"正在處理請求：網址={sheet_url}, 學號={student_id}")
        spreadsheet = gc.open_by_url(sheet_url)
        worksheet = spreadsheet.sheet1
        data = worksheet.get_all_records()
        df = pd.DataFrame(data)

        # 確保 '學號' 欄位是字串型別，以便比對
        if '學號' not in df.columns:
            return None, "錯誤：試算表中找不到 '學號' 欄位。"
        df['學號'] = df['學號'].astype(str)
        student_id = str(student_id)

        student_record = df[df['學號'] == student_id].copy()

        if student_record.empty:
            return None, f"錯誤：在試算表中找不到學號為 '{student_id}' 的學生。"

        # --- 成績計算 ---
        homework_cols = [col for col in df.columns if '作業' in col]
        quiz_cols = [col for col in df.columns if '小測' in col]
        final_exam_col = '期末'
        score_cols = homework_cols + quiz_cols + [final_exam_col]

        for col in score_cols:
            if col in student_record.columns:
                student_record[col] = pd.to_numeric(student_record[col], errors='coerce').fillna(0)

        student_record['平時成績'] = student_record[homework_cols + quiz_cols].mean(axis=1)
        student_record['總成績'] = student_record['平時成績'] * 0.4 + student_record.get(final_exam_col, 0) * 0.6

        display_cols = ['學號'] + [col for col in score_cols if col in student_record.columns] + ['平時成績', '總成績']
        result_df = student_record[display_cols]

        # --- AI 學習建議 ---
        prompt = f"""
        你是一位專業的程式設計課程助教。請根據以下這位學生的成績，提供個人化的學習建議。
        學號：{student_id}
        成績資料：
        {result_df.to_string(index=False)}
        請依據以下結構提供回饋：
        1.  **整體表現總結**
        2.  **優勢分析**
        3.  **待加強部分**
        4.  **具體學習建議**
        請用親切且鼓勵的語氣撰寫。
        """

        print("正在產生 AI 建議...")
        response = model.generate_content(prompt)
        ai_feedback = response.text
        print("✅ 處理完成。")
        return result_df, ai_feedback

    except gspread.exceptions.SpreadsheetNotFound:
        return None, "錯誤：找不到指定的 Google Sheet，請檢查網址是否正確或您是否有權限存取。"
    except Exception as e:
        return None, f"發生未預期的錯誤：{e}"

print("✅ 核心功能函式 get_student_analysis 已成功定義。")

✅ 核心功能函式 get_student_analysis 已成功定義。


In [14]:
# Cell 4: 建立並啟動 Gradio 互動介面
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 AI 成績單分析與個人化學習建議")
    gr.Markdown("請輸入您的 Google Sheet 網址與學號，系統將為您查詢成績並產生 AI 學習建議。")

    with gr.Row():
        sheet_url = gr.Textbox(label="Google Sheet 網址", placeholder="請貼上 Google Sheet 的完整網址...")
        student_id = gr.Textbox(label="學號", placeholder="請輸入要查詢的學號...")

    submit_btn = gr.Button("查詢成績與取得建議", variant="primary")

    gr.Markdown("---")
    gr.Markdown("### 查詢結果")

    student_scores = gr.DataFrame(label="個人成績總覽")

    gr.Markdown("### 🤖 AI 學習建議")
    ai_feedback_output = gr.Markdown(label="個人化建議")

    submit_btn.click(
        fn=get_student_analysis,
        inputs=[sheet_url, student_id],
        outputs=[student_scores, ai_feedback_output]
    )

print("Gradio 介面準備啟動...")
demo.launch(debug=True)

Gradio 介面準備啟動...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://47f031c32409208495.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


正在處理請求：網址=https://docs.google.com/spreadsheets/d/1HA1cvniLmO1O362GtHlrLEK3TQ8_XQJBULMpbaU-29s/edit?usp=sharing, 學號=S011
正在產生 AI 建議...
✅ 處理完成。
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://47f031c32409208495.gradio.live
